In [13]:
import os
import pandas as pd

In [26]:
data_path = "/mnt/data/romy/hypnomed/MRI_raw"
dicom_path = os.path.join(data_path,'DICOM')
bids_path = os.path.join(data_path,'BIDS')

subj_id = 1 #id of the subject for which we want to get the heudiconv infos

In [33]:
heudiconv_path = os.path.join(bids_path,'.heudiconv')
bids_info_paths = list()

for _,sub in enumerate(sorted(os.listdir(heudiconv_path))):
    bids_info_paths.append(os.path.join(heudiconv_path,'{}/ses-001/info/dicominfo_ses-001.tsv'.format(sub))) #permet de récupérer les paths des infos pour chaque sujet
    #bid_tsv = pd.read_csv(bid_path,sep='\t')
    #print(bid_tsv)

bid_tsv = bids_info_paths[subj_id-1] #retrieve the tsv file in which infos for subj_id are stored
print('Getting infos for subject n°{}, stored in {}'.format(subj_id,bid_tsv))
infos = pd.read_csv(bid_tsv,sep='\t')
infos


Getting infos for subject n°1, stored in /mnt/data/romy/hypnomed/MRI_raw/BIDS/.heudiconv/01/ses-001/info/dicominfo_ses-001.tsv


,total_files_till_now,example_dcm_file,series_id,dcm_dir_name,unspecified2,unspecified3,dim1,dim2,dim3,dim4,...,study_description,referring_physician_name,series_description,sequence_name,image_type,accession_number,patient_age,patient_sex,date,series_uid
0,128,1.3.12.2.1107.5.2.43.66012.3000001806040846088...,1-AUTOALIGN,1_AUTOALIGN,-,-,160,160,128,1,...,HUMAINS^NEUCOSE,NaN,AUTOALIGN,*fl3d1_ns,"('ORIGINAL', 'PRIMARY', 'M', 'ND', 'NORM')",ELSES07842,028Y,M,20180604,1.3.12.2.1107.5.2.43.66012.2018060411353876297...
1,133,1.3.12.2.1107.5.2.43.66012.3000001806040846088...,2-AUTOALIGN,2_AUTOALIGN_MPR_sag,-,-,162,162,5,1,...,HUMAINS^NEUCOSE,NaN,AUTOALIGN_MPR_sag,*fl3d1_ns,"('ORIGINAL', 'PRIMARY', 'MPR', 'ND', 'NORM')",ELSES07842,028Y,M,20180604,1.3.12.2.1107.5.2.43.66012.2018060411354435435...
2,136,1.3.12.2.1107.5.2.43.66012.3000001806040846088...,3-AUTOALIGN,3_AUTOALIGN_MPR_cor,-,-,162,162,3,1,...,HUMAINS^NEUCOSE,NaN,AUTOALIGN_MPR_cor,*fl3d1_ns,"('ORIGINAL', 'PRIMARY', 'MPR', 'ND', 'NORM')",ELSES07842,028Y,M,20180604,1.3.12.2.1107.5.2.43.66012.2018060411354435899...
3,139,1.3.12.2.1107.5.2.43.66012.3000001806040846088...,4-AUTOALIGN,4_AUTOALIGN_MPR_tra,-,-,162,162,3,1,...,HUMAINS^NEUCOSE,NaN,AUTOALIGN_MPR_tra,*fl3d1_ns,"('ORIGINAL', 'PRIMARY', 'MPR', 'ND', 'NORM')",ELSES07842,028Y,M,20180604,1.3.12.2.1107.5.2.43.66012.2018060411354436125...
4,158,1.3.12.2.1107.5.2.43.66012.3000001806040846088...,5-LOCA_T1_FL2D_SAG,5_LOCA_T1_FL2D_SAG,-,-,192,192,19,1,...,HUMAINS^NEUCOSE,NaN,LOCA_T1_FL2D_SAG,*fl2d1,"('ORIGINAL', 'PRIMARY', 'M', 'ND', 'NORM')",ELSES07842,028Y,M,20180604,1.3.12.2.1107.5.2.43.66012.2018060411362792554...
5,159,1.3.12.2.1107.5.2.43.66012.3000001806040846088...,7-RUNx_QC_tilt25_AP,7_RUNx_QC_tilt25_AP,-,-,72,74,44,1,...,HUMAINS^NEUCOSE,NaN,RUNx_QC_tilt25_AP,epfid2d1_72,"('ORIGINAL', 'PRIMARY', 'M', 'ND', 'NORM', 'MO...",ELSES07842,028Y,M,20180604,1.3.12.2.1107.5.2.43.66012.2018060411422491167...
6,394,1.3.12.2.1107.5.2.43.66012.3000001806040846088...,8-RUNx_rs_3mmiso_TR2.4_44cpe,8_RUNx_rs_3mmiso_TR2.4_44cpe,-,-,72,74,44,235,...,HUMAINS^NEUCOSE,NaN,RUNx_rs_3mmiso_TR2.4_44cpe,epfid2d1_72,"('ORIGINAL', 'PRIMARY', 'M', 'ND', 'NORM', 'MO...",ELSES07842,028Y,M,20180604,1.3.12.2.1107.5.2.43.66012.2018060411452962891...
7,550,1.3.12.2.1107.5.2.43.66012.3000001806040846088...,10-RUNx_er_3mmiso_TR2.4_44cpe,10_RUNx_er_3mmiso_TR2.4_44cpe,-,-,72,74,44,156,...,HUMAINS^NEUCOSE,NaN,RUNx_er_3mmiso_TR2.4_44cpe,epfid2d1_72,"('ORIGINAL', 'PRIMARY', 'M', 'ND', 'NORM', 'MO...",ELSES07842,028Y,M,20180604,1.3.12.2.1107.5.2.43.66012.2018060411550722034...
8,950,1.3.12.2.1107.5.2.43.66012.3000001806040846088...,12-anat_t2star_2iso,12_anat_t2star_2iso,-,-,108,128,400,1,...,HUMAINS^NEUCOSE,NaN,anat_t2star_2iso,*fl2d5r,"('ORIGINAL', 'PRIMARY', 'M', 'ND', 'NORM')",ELSES07842,028Y,M,20180604,1.3.12.2.1107.5.2.43.66012.2018060412055738118...
9,1350,1.3.12.2.1107.5.2.43.66012.3000001806040846088...,13-anat_t2star_2iso,13_anat_t2star_2iso,-,-,108,128,400,1,...,HUMAINS^NEUCOSE,NaN,anat_t2star_2iso,*fl2d5r,"('ORIGINAL', 'PRIMARY', 'P', 'ND')",ELSES07842,028Y,M,20180604,1.3.12.2.1107.5.2.43.66012.2018060412055738130...
